In [1]:
import pandas as pd
mergedf=pd.read_csv('bladder_cancer/dyngraph2seq_bladder_78/pointOfInterest_bladderCancer_withProperLemma.csv')
significant_words_list=pd.read_csv('bladder_cancer/dyngraph2seq_bladder_78/word_bladder_cancer.idx')['<GO> 1'].str.split(expand=True).iloc[2:,0].to_list()

In [2]:
def applying_wordprocessing(df,column_name,significant_words_list):

    forum_words = []
    selected_words= []
    for index, row in df.iterrows():
        forum_words=forum_words+str(row[column_name]).split()
#     forum_words_merge=' '.join(forum_words)
    for w in forum_words:
        if w in significant_words_list:
            selected_words.append(w)
        
    return selected_words


In [3]:
forum_name_dict={'newly diagnosed': 0, 
                 'non invasive superficial bladder cancer questions and comments': 1, 
                 'muscle invasive bladder cancer questions and comments': 2, 
                 'metastatic bladder cancer': 3, 
                 'women and bladder cancer': 4, 
                 'men and bladder cancer': 5, 
                 'caregivers questions and comments': 6, 
                 'articles of interest': 7, 
                 'chit chat': 8}

In [16]:
# node feature dynamic, forum text user specific for month specific 78 months
import numpy as np
import json
from collections import OrderedDict

input_path = "bladder_cancer/dyngraph2seq_bladder_78/test_wo_post.data"
with open(input_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        jo = json.loads(line, object_pairs_hook=OrderedDict)
        user = jo['user']
        jo['g_ids_features'] = {} # empty dictionary 
        jo['posts'] = {} # empty dictionary
        user_DF = mergedf[mergedf.author.str.match(pat = user,case=True)] # Case sensitive
        
        time_stamp_counter=0
        for yearToMatch in range(2007,2020): 
#             user_DF_year = user_DF[user_DF.year.int.match(pat = yearToMatch,case=True)]
            for monthToMatch in [1,3,5,7,9,11]:  
                mask1 = (user_DF['year'] == yearToMatch) & (user_DF['month'] == monthToMatch)                                                     
                user_DF_month1 = user_DF.loc[mask1]
                month_text_list1 = user_DF_month1['text'].to_list()
                mask2 = (user_DF['year'] == yearToMatch) & (user_DF['month'] == monthToMatch+1)
                user_DF_month2 = user_DF.loc[mask2]
                month_text_list2 = user_DF_month2['text'].to_list()
                month_text_list = month_text_list1+month_text_list2
                if len(month_text_list)==0:
                    jo['posts'][str(time_stamp_counter)] = ' '
                else:      
                    jo['posts'][str(time_stamp_counter)] = ' '.join([str(x) for x in month_text_list])
                result_df = pd.concat([user_DF_month1,user_DF_month2])
                for index,forum_name in enumerate(forum_name_dict):
                    user_DF_perforum = result_df[result_df.forum.str.match(pat = forum_name,case=True)]
                    column_name = 'text'
                    forum_words= applying_wordprocessing(user_DF_perforum,column_name,significant_words_list)
                #   selected_words_dic[index]=set(forum_words)
                    if str(time_stamp_counter) not in jo['g_ids_features']:
                        jo['g_ids_features'][str(time_stamp_counter)]={}
                        if (str(index)) not in jo['g_ids_features'][str(time_stamp_counter)]:
                            jo['g_ids_features'][str(time_stamp_counter)][str(index)]=' '
                            jo['g_ids_features'][str(time_stamp_counter)][str(index)] = jo['g_ids_features'][str(time_stamp_counter)][str(index)]+' '+' '.join(list(set(forum_words)))
                        else:
                             jo['g_ids_features'][str(time_stamp_counter)][str(index)] = jo['g_ids_features'][str(time_stamp_counter)][str(index)]+' '+' '.join(list(set(forum_words)))   
                    else:
                        if (str(index)) not in jo['g_ids_features'][str(time_stamp_counter)]:
                            jo['g_ids_features'][str(time_stamp_counter)][str(index)]=' '
                            jo['g_ids_features'][str(time_stamp_counter)][str(index)] = jo['g_ids_features'][str(time_stamp_counter)][str(index)]+' '+' '.join(list(set(forum_words)))
                        else:
                             jo['g_ids_features'][str(time_stamp_counter)][str(index)] = jo['g_ids_features'][str(time_stamp_counter)][str(index)]+' '+' '.join(list(set(forum_words)))

                time_stamp_counter+=1
 
        filePath = "bladder_cancer/dyngraph2seq_bladder_78/test_dynamic.data"
        with open(filePath, "a+") as f:
            f.write(json.dumps(jo)+"\n")

In [17]:
# making static node features

# tfidf_words=[]
# input_path="bladder_cancer/dyngraph2seq_bladder_78/word_bladder_cancer.idx"
# with open(input_path, 'r') as f:
#     lines = f.readlines()
#     for line in lines[3:]:
#         word = line.strip().split(' ')[0]
#         if word not in tfidf_words:
#             tfidf_words.append(word)
jo_static={}
forum_counter=0
for index,forum_name in enumerate(forum_name_dict):
    user_DF_perforum = mergedf[mergedf.forum.str.match(pat = forum_name,case=True)]
    column_name = 'text'
    forum_words= applying_wordprocessing(user_DF_perforum,column_name,significant_words_list)
    if str(forum_counter) not in jo_static:
        jo_static[str(str(forum_counter))]=' '
        jo_static[str(str(forum_counter))] = jo_static[str(str(forum_counter))]+' '+' '.join(list(set(forum_words)))
    else:
        jo_static[str(str(forum_counter))] = jo_static[str(str(forum_counter))]+' '+' '.join(list(set(forum_words)))
    forum_counter+=1
input_path = "bladder_cancer/dyngraph2seq_bladder_78/test_dynamic.data"
with open(input_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        jo = json.loads(line, object_pairs_hook=OrderedDict)
        jo['g_ids_features'] = jo_static
        filePath = "bladder_cancer/dyngraph2seq_bladder_78/test_static.data"
        with open(filePath, "a+") as f:
            f.write(json.dumps(jo)+"\n")

In [20]:
# static forum data user specific
import numpy as np
import json
from collections import OrderedDict

input_path = "bladder_cancer/dyngraph2seq_bladder_78/train_dynamic.data"
with open(input_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        jo = json.loads(line, object_pairs_hook=OrderedDict)
        user = jo['user']
        jo['g_ids_features'] = {} # empty dictionary 
        user_DF = mergedf[mergedf.author.str.match(pat = user,case=True)] # Case sensitive
        for index,forum_name in enumerate(forum_name_dict):
        #     print(index)
            user_DF_perforum = user_DF[user_DF.forum.str.match(pat = forum_name,case=True)]
            column_name = 'text'
            forum_words= applying_wordprocessing(user_DF_perforum,column_name,significant_words_list)
        #   selected_words_dic[index]=set(forum_words)
            if str(index) not in jo['g_ids_features']:
                jo['g_ids_features'][str(index)]=''
            jo['g_ids_features'][str(index)] = jo['g_ids_features'][str(index)] +' '+' '.join(list(set(forum_words)))
        filePath = "bladder_cancer/dyngraph2seq_bladder_78/train_dyn_user.data"
        with open(filePath, "a+") as f:
            f.write(json.dumps(jo)+"\n")